In [1]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras import Model, Input
from tensorflow.keras.callbacks import EarlyStop, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

ImportError: cannot import name 'EarlyStop' from 'tensorflow.keras.callbacks' (C:\Users\debaj\anaconda3\envs\test\lib\site-packages\keras\api\_v2\keras\callbacks\__init__.py)